In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

print("ステップ1: GitHubからデータを取得")

# より詳細なヘッダーを設定（ブラウザのように見せる）
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Cache-Control': 'max-age=0'
}

# セッションを使用（Cookieなどを保持）
session = requests.Session()
session.headers.update(headers)

# 重複を避けるために辞書で管理
repositories_dict = {}

# Google組織のリポジトリ一覧ページ
base_url = "https://github.com/orgs/google/repositories"
page = 1
max_pages = 94

print(f"ページを開いています: {base_url}\n")

while page <= max_pages:
    # URLを構築
    if page == 1:
        url = f"{base_url}?tab=repositories"
    else:
        url = f"{base_url}?page={page}&tab=repositories"

    try:
        # ページにアクセス
        res = session.get(url, timeout=15)

        # 503エラーの場合はリトライ
        if res.status_code == 503:
            print(f"  503エラー: 待機後に再試行...")
            time.sleep(5)
            res = session.get(url, timeout=15)

        if res.status_code != 200:
            print(f"[ページ {page}] アクセス失敗 (ステータスコード: {res.status_code})")
            break

        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.content, 'html.parser')

        # リポジトリのリスト項目を取得（複数の方法を試す）
        repo_items = soup.find_all('li', class_='ListItem-module__listItem--k4eMk')

        # 別のクラス名も試す
        if not repo_items:
            repo_items = soup.find_all('li', {'data-testid': lambda x: x and 'repository' in str(x).lower()})

        if not repo_items:
            # より広範囲に検索
            repo_items = soup.find_all('li')
            # リポジトリ名を含むものだけフィルタ
            repo_items = [item for item in repo_items if item.find('a', href=lambda x: x and '/google/' in str(x))]

        if not repo_items:
            print(f"[ページ {page}] リポジトリが見つかりません。全ページ読み込み完了\n")
            break

        print(f"Page {page}:")

        page_count = 0
        for item in repo_items:
            # === リポジトリ名を取得 ===
            repo_name = None

            # 方法1: h4タグ内のa
            title_h4 = item.find('h4', class_='Title-module__heading--s7YnL')
            if title_h4:
                repo_link = title_h4.find('a', class_='Title-module__anchor--GmXUE')
                if repo_link:
                    repo_name = repo_link.get_text().strip()

            # 方法2: itemprop="name codeRepository"
            if not repo_name:
                repo_link = item.find('a', {'itemprop': 'name codeRepository'})
                if repo_link:
                    repo_name = repo_link.get_text().strip()

            # 方法3: href="/google/..."のパターン
            if not repo_name:
                repo_link = item.find('a', href=lambda x: x and x.startswith('/google/'))
                if repo_link:
                    repo_name = repo_link.get_text().strip()

            if not repo_name:
                continue

            # 重複チェック
            if repo_name in repositories_dict:
                continue

            # === 主要な言語を取得 ===
            language = "不明"

            # 方法1: ReposListItem-module__Text_4--mkG7R クラス
            language_span = item.find('span', class_='ReposListItem-module__Text_4--mkG7R')
            if language_span:
                language = language_span.get_text().strip()

            # 方法2: itemprop="programmingLanguage"
            if language == "不明":
                language_span = item.find('span', {'itemprop': 'programmingLanguage'})
                if language_span:
                    language = language_span.get_text().strip()

            # 方法3: LanguageCircle の隣のspan
            if language == "不明":
                lang_circle = item.find('div', class_=lambda x: x and 'LanguageCircle' in str(x))
                if lang_circle and lang_circle.parent:
                    next_span = lang_circle.find_next_sibling('span')
                    if next_span:
                        language = next_span.get_text().strip()

            # 方法4: data-testid="repository-lang-stats-graph"
            if language == "不明":
                lang_div = item.find('div', {'data-testid': 'repository-lang-stats-graph'})
                if lang_div and lang_div.parent:
                    lang_span = lang_div.parent.find('span')
                    if lang_span:
                        language = lang_span.get_text().strip()

            # === スターの数を取得 ===
            stars = 0

            # 方法1: aria-labelに"star"を含むaタグ
            star_link = item.find('a', {'aria-label': lambda x: x and 'star' in str(x).lower()})
            if star_link:
                aria_label = star_link.get('aria-label', '')
                # "34 stars" や "1,234 stars" や "2.5k stars" から数字を抽出
                match = re.search(r'([\d,]+\.?\d*)\s*([km]?)\s*star', aria_label.lower())
                if match:
                    star_text = match.group(1).replace(',', '')
                    unit = match.group(2)
                    if unit == 'k':
                        stars = int(float(star_text) * 1000)
                    elif unit == 'm':
                        stars = int(float(star_text) * 1000000)
                    else:
                        try:
                            stars = int(float(star_text))
                        except:
                            stars = 0

            # 方法2: stargazers へのリンクを探す
            if stars == 0:
                star_link = item.find('a', href=lambda x: x and 'stargazers' in str(x))
                if star_link:
                    # リンクのテキストから数字を取得
                    star_text = star_link.get_text().strip().replace(',', '')
                    if star_text:
                        match = re.search(r'([\d,]+\.?\d*)\s*([km]?)', star_text.lower())
                        if match:
                            num = match.group(1)
                            unit = match.group(2) if len(match.groups()) > 1 else ''
                            if unit == 'k':
                                stars = int(float(num) * 1000)
                            elif unit == 'm':
                                stars = int(float(num) * 1000000)
                            else:
                                try:
                                    stars = int(float(num))
                                except:
                                    stars = 0

            # 方法3: octicon-star アイコンの隣のテキスト
            if stars == 0:
                star_svg = item.find('svg', class_='octicon-star')
                if star_svg and star_svg.parent:
                    # 親要素のテキストからアイコン以外の部分を取得
                    star_text = star_svg.parent.get_text().strip()
                    # 数字だけ抽出
                    match = re.search(r'([\d,]+\.?\d*)\s*([km]?)', star_text.lower())
                    if match:
                        num = match.group(1).replace(',', '')
                        unit = match.group(2) if len(match.groups()) > 1 else ''
                        if unit == 'k':
                            stars = int(float(num) * 1000)
                        elif unit == 'm':
                            stars = int(float(num) * 1000000)
                        else:
                            try:
                                stars = int(float(num))
                            except:
                                stars = 0

            # 重複チェックして辞書に追加
            if repo_name not in repositories_dict:
                repositories_dict[repo_name] = (repo_name, language, stars)
                page_count += 1

                # 指定された形式で表示
                print(f"{page_count}. {repo_name}")
                print(f"    言語: {language}")
                print(f"    スター: {stars}")
                print()

        if page_count == 0:
            print(f"  このページでは新しいリポジトリが見つかりませんでした\n")
        else:
            print(f"ページ {page} 完了: {page_count}件取得 (累計: {len(repositories_dict)}件)")

        print("-" * 70)
        print()

        # 次のページへ
        page += 1

        # 【重要】各リクエストの間に必ず1秒待機
        time.sleep(1)

    except requests.exceptions.Timeout:
        print(f"[ページ {page}] タイムアウトエラー")
        page += 1
        time.sleep(1)
        continue
    except Exception as e:
        print(f"[ページ {page}] エラーが発生: {e}")
        page += 1
        time.sleep(1)
        continue

# 辞書からリストに変換
repositories = list(repositories_dict.values())
print(f"取得完了。 {len(repositories)} 個のリポジトリを取得しました")
print(f"総ページ数: {page - 1}")



ステップ1: GitHubからデータを取得
ページを開いています: https://github.com/orgs/google/repositories

Page 1:
1. device-infra
    言語: Java
    スター: 58

2. nsjail
    言語: C++
    スター: 3600

3. XNNPACK
    言語: C
    スター: 2200

4. skia
    言語: C++
    スター: 10000

5. budoux-extension
    言語: TypeScript
    スター: 16

6. nearby
    言語: C++
    スター: 887

7. qwix
    言語: Python
    スター: 64

8. aarch64-rt
    言語: Rust
    スター: 17

9. percore
    言語: Rust
    スター: 8

10. wuffs
    言語: C
    スター: 4700

11. magika
    言語: Python
    スター: 9800

12. jimfs
    言語: Java
    スター: 2500

13. zerocopy
    言語: Rust
    スター: 2100

14. skia-buildbot
    言語: Go
    スター: 158

15. gtm-session-fetcher
    言語: Objective-C
    スター: 265

16. google-toolbox-for-mac
    言語: Objective-C
    スター: 1200

17. ground-android
    言語: Kotlin
    スター: 272

18. command-fds
    言語: Rust
    スター: 42

19. yapf
    言語: Python
    スター: 14000

20. angle
    言語: C++
    スター: 3800

21. tsl
    言語: C++
    スター: 101

22. osv-scalibr
    言語: Go
    スター: 537

23

In [ ]:
# ステップ2: データベースに保存

if repositories:
    print("\nステップ2: データベースに保存")

    conn = sqlite3.connect('github_repositories.db')
    cursor = conn.cursor()

    # 既存のテーブルを削除して新規作成
    cursor.execute('DROP TABLE IF EXISTS repositories')

    # テーブル作成
    cursor.execute('''
    CREATE TABLE repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repository_name TEXT NOT NULL UNIQUE,
        main_language TEXT,
        star_count INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # データ挿入
    cursor.executemany(
        'INSERT INTO repositories (repository_name, main_language, star_count) VALUES (?, ?, ?)',
        repositories
    )
    conn.commit()

    print(f"データベースに {len(repositories)} 件保存しました")
    print(f"データベース名: github_repositories.db")
    print(f"テーブル名: repositories")

   
    # ステップ3: 保存したデータをSELECT文で表示
    
    print("\nステップ3: 保存したデータの確認")
    

    # 全データ件数を確認
    cursor.execute('SELECT COUNT(*) FROM repositories')
    total_count = cursor.fetchone()[0]
    print(f"\n総レコード数: {total_count}件\n")

    # 最初の20件を表示
    print("【SELECT文実行】最初の20件:")
    print("-" * 70)
    cursor.execute('''
        SELECT id, repository_name, main_language, star_count
        FROM repositories
        ORDER BY id
        LIMIT 20
    ''')

    results = cursor.fetchall()
    print(f"{'ID':<5} {'リポジトリ名':<40} {'言語':<15} {'スター数':>10}")
    print("-" * 70)
    for row in results:
        print(f"{row[0]:<5} {row[1]:<40} {row[2]:<15} {row[3]:>10,}")

    # スター数トップ10
    print("\n" + "=" * 70)
    print("「SELECT文実行」スター数トップ10:")
    print("-" * 70)
    cursor.execute('''
        SELECT repository_name, main_language, star_count
        FROM repositories
        ORDER BY star_count DESC
        LIMIT 10
    ''')

    results = cursor.fetchall()
    print(f"{'順位':<5} {'リポジトリ名':<40} {'言語':<15} {'スター数':>10}")
    print("-" * 70)
    for i, row in enumerate(results, 1):
        print(f"{i:<5} {row[0]:<40} {row[1]:<15} {row[2]:>10,}")

    # 言語別集計
    print("「SELECT文実行」言語別リポジトリ数:")
    cursor.execute('''
        SELECT main_language, COUNT(*) as count
        FROM repositories
        GROUP BY main_language
        ORDER BY count DESC
        LIMIT 15
    ''')

    results = cursor.fetchall()
    print(f"{'言語':<20} {'リポジトリ数':>10}")
    print("-" * 70)
    for row in results:
        print(f"{row[0]:<20} {row[1]:>10}")

    conn.close()
    print("すべての処理が完了しました。")
else:
    print("\n取得したデータがありません。")


ステップ2: データベースに保存
データベースに 2812 件保存しました
データベース名: github_repositories.db
テーブル名: repositories

ステップ3: 保存したデータの確認

総レコード数: 2812件

【SELECT文実行】最初の20件:
----------------------------------------------------------------------
ID    リポジトリ名                                   言語                    スター数
----------------------------------------------------------------------
1     device-infra                             Java                    58
2     nsjail                                   C++                  3,600
3     XNNPACK                                  C                    2,200
4     skia                                     C++                 10,000
5     budoux-extension                         TypeScript              16
6     nearby                                   C++                    887
7     qwix                                     Python                  64
8     aarch64-rt                               Rust                    17
9     percore                                  